 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
Tangi Migot et Paul Raynaud

## Réaliser par : `Taha El Arrouchi`
Matricule : `2194007`

### Exercice 1: Newton avec recherche linéaire - amélioration du code

In [1]:
using Pkg
Pkg.activate(".") #Accède au fichier Project.toml
Pkg.instantiate()
Pkg.add("LDLFactorizations")
Pkg.add("OptimizationProblems")
Pkg.add("Ipopt")
Pkg.add("JuMP")
Pkg.status()


  Activating project at `c:\Users\Admin\.github\MTH8408-Hiv24`


   Resolving package versions...

  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Project.toml`
  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Manifest.toml`

   Resolving package versions...

  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Project.toml`
  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Manifest.toml`

   Resolving package versions...

  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Project.toml`
  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Manifest.toml`

   Resolving package versions...

  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Project.toml`
  No Changes to `C:\Users\Admin\.github\MTH8408-Hiv24\Manifest.toml`


Status `C:\Users\Admin\.github\MTH8408-Hiv24\Project.toml`
  [54578032] ADNLPModels v0.7.0
  [f6369f11] ForwardDiff v0.10.36
  [b6b21f68] Ipopt v1.6.1
  [4076af6c] JuMP v1.19.0
  [40e66cde] LDLFactorizations v0.10.1
  [a4795742] NLPModels v0.20.0
⌃ [5049e819] OptimizationProblems v0.5.0
Info Packages marked with ⌃ have new versions available and may be upgradable.

In [2]:
using ADNLPModels, LinearAlgebra, NLPModels, Printf

In [3]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end


armijo (generic function with 1 method)

In [4]:
function newton_armijo(nlp, x0; verbose::Bool = true)
    F(x) = obj(nlp, x)
    g(x) = grad(nlp, x)
    H(x) = hess(nlp, x)
    xk  = x0
    fk  = F(xk)
    gk = g(xk)
    gnorm = gnorm0 = norm(gk)
    k = 0
    verbose  && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
    verbose  && @printf "%2d %9.2e %9.1e\n" k fk gnorm
    evaluations = 0 # Apres 1000 évaluation, on arrête
    t_0=time() # temps de départ, on arrête après 60s
    while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
      Hk = H(xk)
      dk = - Hk \ gk
      slope = dot(dk, gk)
      λ = 0.0
      itt = 0 # Apres 5 ittération, on passe
      while slope ≥ -1.0e-4 * norm(dk) * gnorm
        λ = max(1.0e-3, 10 * λ)
        dk = - ((Hk + λ * I ) \ gk)
        slope = dot(dk, gk)
        itt += 1
        println("Ittération numéro:", itt)
        if itt == 5
          dk = -gk
          break
        end
      end
      t = armijo(xk, dk, fk, gk, F)
      xk += t * dk
      fk = F(xk)
      if fk < -1e15
        println("le problème est non-bornée inférieurement.")
        break
      end
      gk = g(xk)
      gnorm = norm(gk)
      k += 1
      verbose && @printf "%2d %9.2e %9.1e %7.1e\n" k fk gnorm t

      evaluations += neval_obj(nlp)
      t_f = time() - t_0
      println("Nombre d'évaluation: ", evaluations," Temps d'évaluation: ", t_f)
      if evaluations >= 1000
        println("Nombre maximal d'évaluations de fonctions est atteint.")
        break
      end
    if t_f >= 60
        println("Limite de temps atteinte.")
        break
    end
    end
    return xk
  end


newton_armijo (generic function with 1 method)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

In [5]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0=[.5, .5]
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||


 0  1.00e+00   4.5e+00


 1  5.08e-02   8.4e-01 1.0e+00
Nombre d'évaluation: 3 Temps d'évaluation: 1.503000020980835
 2  4.73e-04   7.6e-02 1.0e+00
Nombre d'évaluation: 8 Temps d'évaluation: 1.5370001792907715
 3  6.97e-08   9.1e-04 1.0e+00
Nombre d'évaluation: 15 Temps d'évaluation: 1.5370001792907715
 4  1.62e-15   1.4e-07 1.0e+00
Nombre d'évaluation: 24 Temps d'évaluation: 1.5370001792907715


2-element Vector{Float64}:
 2.323057366509693e-8
 2.3230573665124037e-8

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$
Ce type de factorisation n'est possible que si la matrice hessienne est définie positive, dans le cas contraire on a besoin de régularisé le système comme dans l'exercice précédent.

Pour résoudre le système linéaire en utilisant cette factorisation, on va utiliser le package [`LDLFactorizations`](https://github.com/JuliaSmoothOptimizers/LDLFactorizations.jl):

In [6]:
using LDLFactorizations, LinearAlgebra

#### Utiliser de la correction pour calculer la direction de descente:

In [7]:
# Solution: modifier le calcul de la direction avec LDLFactorizations
function newton_ldlt_armijo(nlp, x0; verbose::Bool = true)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = Symmetric(triu(hess(nlp, xk)), :U)
    Sk = ldl_analyze(Hk)
    ldl_factorize!(Hk, Sk)
    Sk.d=abs.(Sk.d)
    dk = - Sk \ gk
    slope = dot(dk, gk)
    t = armijo(xk, dk, fk, gk, x -> obj(nlp, x))
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_ldlt_armijo (generic function with 1 method)

In [8]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))

newton_ldlt_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00


 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


2-element Vector{Float64}:
 2.323057366856638e-8
 2.3230573665124037e-8

### Exercice 3: Méthode quasi-Newton: BFGS

In [9]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec la méthode de BFGS inverse skip.
function bfgs_quasi_newton_armijo(nlp, x0; verbose::Bool = true)
  xk = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  Hk=zeros(2,2,100)
  yk=zeros(2,100) 
  rhok=zeros(1,100) 
  # J'utiliserai un pas de 1/10 pour les x
  sk=ones(2)*1/10
  for i in 1:100
  yk[:,i]=grad(nlp, i*sk)-grad(nlp, (i-1)*sk)
  rhok[i] = 1 / (transpose(yk[:, i]) * sk)
  end
  #Hk(:,:,1) c'est H_0 
  Hk[:,:,1]=(I-rhok[1]*sk*transpose(yk[:,1]))*(I-rhok[1]*yk[:,1]*transpose(sk))+rhok[1]*sk*transpose(sk)
  #Hk(:,:,1)=(transpose(yk[:,1])*sk)/(transpose(yk[:,1])*yk[:,1])*I
  for i in 2:100
  Hk[:,:,i]=(I-rhok[i]*sk*transpose(yk[:,i]))*Hk[:,:,i-1]*(I-rhok[i]*yk[:,i]*transpose(sk))+rhok[i]*sk*transpose(sk)
  end
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    # on prendra la valeur de xk=[0.4 0.4], donc c'est H(5) 
    Hkk = Hk[:,:,5]
    dk = - Hkk*gk
    slope = dot(dk, gk)
    t = armijo(xk, dk, fk, gk, x -> obj(nlp, x))
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e\n" k fk gnorm t
  end
  return xk
end

bfgs_quasi_newton_armijo (generic function with 1 method)

In [10]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))
bfgs_quasi_newton_armijo(nlp, [0.4,0.4])


 k        fk ||∇f(x)||
 0  6.08e-01   3.4e+00
 1  3.56e-02   7.0e-01 1.0e+00
 2  5.99e-03   2.8e-01 1.0e+00
 3  1.20e-03   1.2e-01 1.0e+00
 4  2.55e-04   5.6e-02 1.0e+00
 5  5.58e-05   2.6e-02 1.0e+00
 6  1.24e-05   1.2e-02 1.0e+00
 7  2.77e-06   5.8e-03 1.0e+00
 8  6.19e-07   2.7e-03 1.0e+00
 9  1.39e-07   1.3e-03 1.0e+00
10  3.11e-08   6.1e-04 1.0e+00
11  6.98e-09   2.9e-04 1.0e+00
12  1.57e-09   1.4e-04 1.0e+00
13  3.51e-10   6.5e-05 1.0e+00
14  7.88e-11   3.0e-05 1.0e+00
15  7.53e-12   1.2e-05 1.0e+00
16 -1.71e-09   2.0e-04 1.0e+00
17 -2.90e-07   2.6e-03 1.0e+00
18 -4.92e-05   3.4e-02 1.0e+00
19 -8.64e-03   4.7e-01 1.0e+00
20 -2.30e+00   1.0e+01 1.0e+00
21 -8.25e+03   2.2e+03 1.0e+00
22 -6.00e+10   8.2e+07 1.0e+00
23 -3.11e+24   1.1e+17 1.0e+00


2-element Vector{Float64}:
 -1.158144643028232e8
  6.472267542659859e-9

### Exercice 4: application à un problème de grande taille

On va ajouter le package `OptimizationProblems` qui contient, comme son nom l'indique, une collection de problème d'optimisation disponible au format de `JuMP` (dans le sous-module `OptimizationProblems.PureJuMP`) et de `ADNLPModel` (dans le sous-module `OptimizationProblems.ADNLPProblems`).

In [13]:
using ADNLPModels, OptimizationProblems.ADNLPProblems # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!
using OptimizationProblems

In [14]:
# Use previous functions to solve genrose.

# Générer le problème genrose
n = 500
model = genrose(n=n)

# Vérifier s'il est sans contrainte
is_unconstrained = unconstrained(model)

# Afficher le résultat
println("Le problème genrose est sans contrainte : ", is_unconstrained)

# Accéder aux informations sur le problème en utilisant son meta
meta = OptimizationProblems.genrose_meta
println("Informations sur le problème genrose :")
println(meta)
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
# Résoudre le problème genrose
m = Model(Ipopt.Optimizer)
@variable(m, x[1:n])
@NLobjective(m, Min, x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1))

optimize!(m)

# Afficher la solution
println("Solution du problème genrose :")
println(value.(x))

Le problème genrose est sans contrainte : true
Informations sur le problème genrose :


Dict{Symbol, Any}(

:has_equalities_only => false, :origin => :unknown, :has_inequalities_only => false, :defined_everywhere => missing, :has_fixed_variables => false, :variable_ncon => false, :nvar => 100, :is_feasible => true, :minimize => true, :ncon => 0, :name => "genrose", :best_known_lower_bound => -Inf, :objtype => :other, :best_known_upper_bound => 405.1064193957891, :has_bounds => false, :variable_nvar => true, :contype => :unconstrained)

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,